In [1]:
import numpy as N
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from datetime import datetime,timedelta
import pytz as pytz
import sys
import os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import modules.thermolib as thermo
import modules.DSDlib as dsd
import modules.disdrometer_module as dis
import pandas as pd
import glob


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119 


In [2]:
dis_dir = '/Users/ddawson/Dropbox/PIPS_data/processed/061416_061616/'
dis_files = glob.glob(dis_dir+'*.txt')
print dis_files

['/Users/ddawson/Dropbox/PIPS_data/processed/061416_061616/PIPS_1A_061416_061616_D1.txt', '/Users/ddawson/Dropbox/PIPS_data/processed/061416_061616/PIPS_1B_061416_061616_D1.txt', '/Users/ddawson/Dropbox/PIPS_data/processed/061416_061616/PIPS_2A_061416_061616_D1.txt', '/Users/ddawson/Dropbox/PIPS_data/processed/061416_061616/PIPS_2B_061416_061616_D1.txt']


In [ ]:
fasttemps_series_list = []
for dis_file in dis_files:
    datetimesUTC,pdatetimesUTC,flaggedtimes,intensities,preciptots,reflectivities,pcounts,pcounts2, \
           sensortemps,concentrations,onedrop_concentrations,countsMatrix,windspds,winddirrels,winddirabss, \
        winddiags,fasttemps,slowtemps,dewpoints,RHs_derived,RHs,pressures,compass_dirs,    \
        GPS_lats,GPS_lons,GPS_stats,GPS_alts,voltages,DSD_interval,DSD_intervalstr,DSD_index = dis.readPIPS(dis_file,rainonlyqc=False,DSD_interval=10.0)

    DSD_interval_td = timedelta(seconds=DSD_interval)
    DSD_halfinterval_td = timedelta(seconds=DSD_interval/2.)

    # Create a pandas series from the fasttemps
    
    fasttemps_series = pd.Series(fasttemps,index=datetimesUTC)
    fasttemps_series_list.append(fasttemps_series)

# Now, join the separate series into a single Dataframe, using only the common times for each.

fasttemps_df = pd.concat(fasttemps_series_list,join='inner',axis=1)
print fasttemps_df

140616 133028
2016-06-14 13:30:28
2016 6 14 13 30 28
2016-06-14 13:30:28 2016-06-14 13:30:28
GPS Offset 0:00:00
Requested DSD interval: 10.0. Actual DSD interval: 10.0
140616 133628
2016-06-14 13:36:28
2016 6 14 13 36 28
2016-06-14 13:36:28 2016-06-14 13:36:28
GPS Offset 0:00:00
Requested DSD interval: 10.0. Actual DSD interval: 10.0
140616 134357
2016-06-14 13:43:57
2016 6 14 13 43 57
2016-06-14 13:43:57 2016-06-14 13:43:57
GPS Offset 0:00:00
Requested DSD interval: 10.0. Actual DSD interval: 10.0
140616 134834
2016-06-14 13:48:34
2016 6 14 13 48 34
2016-06-14 13:48:34 2016-06-14 13:48:34
GPS Offset 0:00:00


In [ ]:
# Scratch code below, don't execute!
    
    # Convert to numpy arrays
    windspds = N.array(windspds)
    winddirrels = N.array(winddirrels)
    winddirabss = N.array(winddirabss)
    fasttemps = N.array(fasttemps)
    slowtemps = N.array(slowtemps)
    dewpoints = N.array(dewpoints)
    RHs_derived = N.array(RHs_derived)
    RHs = N.array(RHs)
    pressures = N.array(pressures)
    pcounts = N.array(pcounts)
    pcounts2 = N.array(pcounts2)
    intensities = N.array(intensities)

    Nc_bin = concentrations.T
    dropperbin = onedrop_concentrations.T

    logNc_bin = N.ma.log10(Nc_bin)
    logNc_bin = N.ma.masked_where(Nc_bin < dropperbin, logNc_bin)

    # Compute potential temperature, water vapor mixing ratio, and density
    pt = thermo.caltheta(pressures*100.,slowtemps+273.15)
    qv = thermo.calqv(RHs/100.,pressures*100.,slowtemps+273.15)
    rho=thermo.calrho(pressures*100.,pt,qv)